In [1]:
import sqlite3

In [2]:
# Connect to a new SQLite database file
conn = sqlite3.connect('../sample_crm.db')
cur = conn.cursor()

In [3]:
# Enable foreign key support
cur.execute("PRAGMA foreign_keys = ON;")

In [4]:
# Drop existing tables (for repeatable runs)
cur.executescript("""
DROP TABLE IF EXISTS transactions;
DROP TABLE IF EXISTS opportunities;
DROP TABLE IF EXISTS accounts;
""")

In [5]:
# Create tables
cur.executescript("""
CREATE TABLE accounts (
    account_id INTEGER PRIMARY KEY,
    account_name TEXT NOT NULL,
    industry TEXT,
    region TEXT,
    created_at TEXT DEFAULT CURRENT_TIMESTAMP
);

CREATE TABLE opportunities (
    opportunity_id INTEGER PRIMARY KEY,
    account_id INTEGER NOT NULL,
    funnel_value REAL NOT NULL,
    stage TEXT,
    owner TEXT,
    created_at TEXT DEFAULT CURRENT_TIMESTAMP,
    expected_close TEXT,
    FOREIGN KEY (account_id) REFERENCES accounts(account_id)
);

CREATE TABLE transactions (
    account_id INTEGER NOT NULL,
    opportunity_id INTEGER,
    booking_date TEXT NOT NULL,
    amount REAL NOT NULL,
    currency TEXT,
    channel TEXT,
    product TEXT,
    region TEXT,
    FOREIGN KEY (account_id) REFERENCES accounts(account_id),
    FOREIGN KEY (opportunity_id) REFERENCES opportunities(opportunity_id)
);
""")

In [6]:
# Insert data into accounts
accounts = [
    (1, 'Acme Corp', 'Manufacturing', 'North America'),
    (2, 'Beta Ltd', 'Healthcare', 'Europe'),
    (3, 'Gamma GmbH', 'Technology', 'Europe'),
    (4, 'Delta Inc', 'Finance', 'North America')
]
cur.executemany("INSERT INTO accounts (account_id, account_name, industry, region) VALUES (?, ?, ?, ?);", accounts)

# Insert data into opportunities
opportunities = [
    (101, 1, 25000.0, 'Proposal', 'Alice', '2025-08-15'),
    (102, 1, 50000.0, 'Negotiation', 'Bob', '2025-09-01'),
    (103, 2, 15000.0, 'Qualified', 'Charlie', '2025-08-30'),
    (104, 3, 70000.0, 'Closed Won', 'Diana', '2025-07-01')
]
cur.executemany("""
INSERT INTO opportunities (opportunity_id, account_id, funnel_value, stage, owner, expected_close)
VALUES (?, ?, ?, ?, ?, ?);
""", opportunities)

# Insert data into transactions
transactions = [
    (1, 101, '2025-07-15', 1000.0, 'USD', 'Online', 'Subscription', 'North America'),
    (1, 101, '2025-07-20', 2000.0, 'USD', 'Partner', 'One-time', 'North America'),
    (1, None, '2025-07-25', 300.0, 'USD', 'Online', 'Add-on', 'North America'),
    (2, 103, '2025-07-18', 800.0, 'EUR', 'Online', 'Consulting', 'Europe'),
    (2, None, '2025-07-22', 1200.0, 'EUR', 'Reseller', 'Hardware', 'Europe'),
    (3, 104, '2025-07-01', 5000.0, 'EUR', 'Direct', 'Software', 'Europe'),
    (3, 104, '2025-07-05', 2000.0, 'EUR', 'Direct', 'Maintenance', 'Europe'),
    (4, None, '2025-07-10', 1500.0, 'USD', 'Online', 'Advisory', 'North America')
]
cur.executemany("""
INSERT INTO transactions (account_id, opportunity_id, booking_date, amount, currency, channel, product, region)
VALUES (?, ?, ?, ?, ?, ?, ?, ?);
""", transactions)

In [7]:
# Commit and close connection
conn.commit()
conn.close()